In [1]:

import os
from traceback import print_tb
import pandas as pd
import duckdb

import os, sys 
from pathlib import Path
tmp = Path.cwd()
print(tmp)
os.chdir(Path.cwd().parent)


if Path.cwd() not in sys.path:
 sys.path.append(Path.cwd())

import utils


os.chdir(tmp)
#print(tmp)


/Users/miona.dimic/Desktop/MDS/ADSDB/project/mimi-adsdb/Notebooks/4.Exploitation zone


### ETL household table

In [2]:
con = duckdb.connect('../household.duckdb')

dfH = con.execute("SELECT *  FROM household").df()
con.close()




dfH['section'] = dfH.section.apply(lambda s: s.strip()[:-2] if s.strip()[-2:]=='.0' else s)
dfH['section'] = dfH.section.apply(lambda s: '0' + s if len(s)==4  else s)
dfH['section'] = dfH.section.apply(lambda s: '0796' + s if len(s)== 5 else s)
dfH['section'] = dfH.section.apply(lambda s: '0796' if s.strip()[-6:]=='Madrid' else s)
dfH['section'] = dfH.section.apply(lambda s: '0796' + s.strip()[0:2] if s.strip()[2]=='.'else s)




dfH['section'] = dfH['section'].str.replace(' ', '') # remove extra spaces


dfH = dfH.rename(columns = {'section':'Madrid_section', 'habitantes':'suma de categorías de hogares'})  #rename in right format

utils.df_to_DBtable(f'../integration.duckdb',dfH, 'householdClean_Madrid')

display(dfH)

,Madrid_section,suma de categorías de hogares,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,...,two_adults_over_35_and_one_adult_from_16_to_34_and_two_minors,three_adults_and_0_or_more_minors,two_adults_over_35_and_two_adults_from_16_to_34,two_adults_over_35_and_two_adults_from_16_to_34_and_one_minor,two_adults_over_35_and_two_adults_from_16_to_34_and_two_or_more_minors,four_adults_and_0_or_more_minors,five_adults_and_0_or_more_minors,fifteen_or_more_inhabitants,only_minors,Year
0,0796,1290164.0,117454.0,115691.0,126044.0,36720.0,27171.0,5491.0,158487.0,174284.0,...,9057.0,104079.0,51419.0,8009.0,3438.0,46977.0,53887.0,552.0,405.0,2019
1,079601,67296.0,11627.0,13352.0,5117.0,2171.0,922.0,194.0,12698.0,5244.0,...,214.0,4341.0,1062.0,184.0,84.0,1711.0,1934.0,44.0,33.0,2019
2,079601001,559.0,67.0,88.0,62.0,16.0,9.0,4.0,101.0,69.0,...,1.0,35.0,11.0,4.0,2.0,13.0,17.0,0.0,1.0,2019
3,079601002,471.0,99.0,89.0,36.0,15.0,5.0,3.0,84.0,29.0,...,0.0,41.0,11.0,1.0,0.0,13.0,12.0,0.0,0.0,2019
4,079601003,855.0,147.0,151.0,71.0,42.0,7.0,1.0,144.0,75.0,...,2.0,54.0,16.0,5.0,1.0,20.0,35.0,0.0,2.0,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7390,079621029,519.0,32.0,29.0,6.0,5.0,20.0,12.0,45.0,19.0,...,10.0,29.0,49.0,7.0,2.0,8.0,12.0,0.0,0.0,2018
7391,079621030,566.0,37.0,46.0,29.0,11.0,19.0,6.0,68.0,52.0,...,3.0,39.0,41.0,3.0,0.0,14.0,16.0,0.0,0.0,2018
7392,079621031,887.0,88.0,91.0,15.0,6.0,39.0,9.0,156.0,31.0,...,6.0,28.0,21.0,6.0,2.0,10.0,16.0,0.0,0.0,2018
7393,079621032,627.0,65.0,71.0,17.0,4.0,27.0,13.0,82.0,23.0,...,2.0,21.0,30.0,5.0,0.0,7.0,6.0,1.0,1.0,2018


### ETL Nationalities

In [3]:
# extract only those related to Madrid municipality
con = duckdb.connect('../nationalities.duckdb')
cleanNatTable = con.execute(f'SELECT * FROM nationalitiesClean').df()
con.close()
cleanNatTable = cleanNatTable[cleanNatTable['Madrid_section'].str.contains('0796', regex=False)]

cleanNatTable = cleanNatTable.rename(columns = {'Habitantes':'suma de categorías de nacionalidad'})  #rename in right format

cleanNatTable['Madrid_section'] = cleanNatTable['Madrid_section'].str.replace(' ', '') # remove extra spaces

utils.df_to_DBtable('../integration.duckdb',cleanNatTable, 'nationalitiesClean_Madrid')


display(cleanNatTable)

,Afganistán,Albania,Alemania,Andorra,Angola,Antigua_y_Barbuda,Arabia_Saudita,Argelia,Argentina,Armenia,...,Uganda,Uruguay,Uzbekistán,Vanuatu,Venezuela,Vietnam,Year,Yemen,Zambia,Zimbabwe
1271,110.0,272.0,4645.0,18.0,216.0,2.0,247.0,1162.0,4933.0,561.0,...,17.0,901.0,64.0,0.0,19366.0,251.0,2018,33.0,2.0,18.0
1272,12.0,23.0,606.0,2.0,5.0,0.0,7.0,89.0,585.0,22.0,...,2.0,86.0,3.0,0.0,1447.0,22.0,2018,0.0,0.0,0.0
1273,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,6.0,0.0,2018,0.0,0.0,0.0
1274,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,3.0,0.0,0.0,9.0,0.0,2018,0.0,0.0,0.0
1275,0.0,0.0,5.0,0.0,0.0,0.0,0.0,2.0,15.0,0.0,...,0.0,5.0,0.0,0.0,21.0,0.0,2018,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13413,0.0,0.0,2.0,0.0,5.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,2020,0.0,0.0,0.0
13414,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,19.0,0.0,2020,0.0,0.0,0.0
13415,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,40.0,0.0,2020,0.0,0.0,0.0
13416,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,8.0,0.0,2020,0.0,0.0,0.0


In [46]:
# check if clean tables are in integration.duckdb are created
con = duckdb.connect('../integration.duckdb')
int_tables = con.execute(f'SELECT table_name FROM information_schema.tables').df()
con.close()
int_tables

,table_name
0,householdClean_Madrid
1,nationalitiesClean_Madrid


In [4]:
con = duckdb.connect('../integration.duckdb')
nationalites = con.execute(f'SELECT * FROM nationalitiesClean_Madrid').df()
households = con.execute(f'SELECT * FROM householdClean_Madrid').df()
con.close()



# # checking if all section codes are same in both tables

sectionsH = households['Madrid_section'].tolist()
sectionsN = nationalites['Madrid_section'].tolist()


difference = [item for item in sectionsH if item not in sectionsN]
# if difference is 0 => both tables have same set of codes for Madrid section

print(difference)
print(len(difference))

[]
0
